In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import pnn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo

In [18]:
def gen_pattern_replace_and_matcher_for_loop_pnn_mul(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None, device = 'cpu',dim = 32
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          return self.pn(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total

    def pn(self,embed):
      empty_tensor = torch.empty((batch,self.total,dim),device=device)
      row = embed[getitem_node_args[0]] 
      col = embed[getitem_node_args[1]]
      empty_tensor[:,self.num_prefix:] = row[0,self.num_prefix:] * col[0,self.num_prefix:]
      empty_tensor[:,:self.num_prefix] = row[:,:self.num_prefix] * col[:,:self.num_prefix]
      return torch.sum(empty_tensor,dim = -1)


    def forward(self,x):
      return self.pn(x)
    
  
  return pattern,ReplacementClass(),_match

In [14]:
# 只修改哈达玛积
def workload_pnn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096,device = 'cuda'):
  print(f"now gen workload of PNN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(num_field)],dim,l,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(num_field)],dim,l,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn_mul(pnn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch,dim=dim,device=device)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [19]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_pnn,l = [1024,512,256]):
  def run(model):
    model = model.cuda()
    model.eval()
    # traced_model = torch.jit.trace(, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda())
    compiled_model = torch.compile(model, backend="inductor")
    compiled_model.eval()
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long).cuda()
    for i in range(6):
        soutput = compiled_model(t)
    torch.cuda.synchronize()
    start_time = time.time()  # 开始计时

    for i in range(100):
        soutput = compiled_model(t)
    # torch.cuda.synchronize()
    end_time = time.time()  # 结束计时
        
        # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    print(f"time : {elapsed_time * 1000 /100} ms")
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l,batch = batch,device = 'cuda')
  run(ori)
  run(modify)

In [22]:
gen_and_test(num_field = 22 ,prefix = 10 , batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0_1 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant1 target _tensor_constant1 _tensor_constant1 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(


time : 0.3384065628051758 ms
time : 0.4471778869628906 ms


In [24]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0_1 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant1 target _tensor_constant1 _tensor_constant1 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(


time : 2.6438069343566895 ms
time : 8.160557746887207 ms
